# Data analysis and price prediction: Airbnb listings in Berlin
### Dataset
Is accessible as 'Airbnb Berlin.csv', retrieved from kaggle.
### Objective
Build a predictive model: Predict price based on features of a listing such as location, size etc. Prediction shall be understood as a helping tool for new listings to estimate appropriate price. Therefore, reviews and ratings of listings are in this step irrelevant.

In [2]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Data exploration and cleaning

In the first steps the data set is simplified by:
* Dropping duplicates in Listing ID (as duplicates should contain same information per listing)
* Selecting only features (columns) relevant for price prediction

In [6]:
# read csv and print info
df = pd.read_csv('Airbnb_Berlin.csv', dtype={'Postal Code': 'str', 'Price': 'str'})
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456961 entries, 0 to 456960
Data columns (total 47 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   index                  456961 non-null  int64  
 1   Review ID              452805 non-null  float64
 2   review_date            452805 non-null  object 
 3   Reviewer ID            452805 non-null  float64
 4   Reviewer Name          452805 non-null  object 
 5   Comments               452597 non-null  object 
 6   Listing ID             456961 non-null  int64  
 7   Listing URL            456961 non-null  object 
 8   Listing Name           456756 non-null  object 
 9   Host ID                456961 non-null  int64  
 10  Host URL               456961 non-null  object 
 11  Host Name              456913 non-null  object 
 12  Host Since             456913 non-null  object 
 13  Host Response Time     398194 non-null  object 
 14  Host Response Rate     398194 non-nu

In [5]:
# print head
print(df.head())

   index   Review ID review_date  Reviewer ID Reviewer Name  \
0      0  58660447.0    01-03-16   14876562.0        Dennis   
1      1         NaN         NaN          NaN           NaN   
2      2  58663795.0    01-03-16   49367757.0       Andreea   
3      3  59003619.0    01-05-16   46694292.0         Sasha   
4      4  66845661.0    03-25-16   54542819.0        Yohana   

                                            Comments  Listing ID  \
0  Very nice, clean appartement! Josh was really ...    10002699   
1                                                NaN    10002922   
2  Aki is really nice and friendly. We had an ama...    10006647   
3  Aki is a very nice dude with a clean place in ...    10006647   
4  Aki's place was easy to find and located in th...    10006647   

                             Listing URL                     Listing Name  \
0  https://www.airbnb.com/rooms/10002699  Trendy and peaceful KreuzkÃ¶lln   
1  https://www.airbnb.com/rooms/10002922  Sehr zentrale Wo

In [7]:
# print number of unique values in listing ID
print(df['Listing ID'].nunique())

23536


In [13]:
# drop duplicates in Listing ID
df = df.drop_duplicates(subset='Listing ID')

# drop columns concerning reviews 
col_reviews_ratings = ['Review ID', 'review_date', 'Reviewer ID', 'Reviewer Name', 'Comments', 'Overall Rating', 'Reviews', 
                       'First Review', 'Last Review', 'Accuracy Rating', 'Cleanliness Rating', 'Checkin Rating', 
                       'Communication Rating', 'Location Rating', 'Value Rating']
df = df.drop(labels=col_reviews_ratings, axis=1)

# again print info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23536 entries, 0 to 456958
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  23536 non-null  int64  
 1   Listing ID             23536 non-null  int64  
 2   Listing URL            23536 non-null  object 
 3   Listing Name           23482 non-null  object 
 4   Host ID                23536 non-null  int64  
 5   Host URL               23536 non-null  object 
 6   Host Name              23515 non-null  object 
 7   Host Since             23515 non-null  object 
 8   Host Response Time     13046 non-null  object 
 9   Host Response Rate     13046 non-null  object 
 10  Is Superhost           23515 non-null  object 
 11  neighbourhood          23536 non-null  object 
 12  Neighborhood Group     23536 non-null  object 
 13  City                   23533 non-null  object 
 14  Postal Code            22951 non-null  object 
 15  C

Furthermore, we drop:
* columns that contain other irrelevant aspects such as URLs, IDs (except Listing ID for now), names, host related info (except Is Superhost).
* column wih very low non-null count: Square Feet

In [14]:
# drop descriptive columns  
col_irr = ['Listing URL', 'Listing Name', 'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Response Time', 
           'Host Response Rate', 'Square Feet']
df = df.drop(labels=col_irr, axis=1)

# again print info
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23536 entries, 0 to 456958
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  23536 non-null  int64  
 1   Listing ID             23536 non-null  int64  
 2   Is Superhost           23515 non-null  object 
 3   neighbourhood          23536 non-null  object 
 4   Neighborhood Group     23536 non-null  object 
 5   City                   23533 non-null  object 
 6   Postal Code            22951 non-null  object 
 7   Country Code           23536 non-null  object 
 8   Country                23536 non-null  object 
 9   Latitude               23536 non-null  float64
 10  Longitude              23536 non-null  float64
 11  Is Exact Location      23536 non-null  object 
 12  Property Type          23536 non-null  object 
 13  Room Type              23536 non-null  object 
 14  Accomodates            23536 non-null  int64  
 15  B